In [ ]:
#Ativar o autocomplete
%config IPCompleter.greedy=True

In [ ]:
import json
import pandas as pd
from functools import reduce
import os
from IPython.display import display, HTML
from pathlib import Path
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ctypes

In [ ]:
def display_df(df, show_index=True):
    display(HTML(df.to_html(index=show_index)))

In [ ]:
def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

In [ ]:
def list_append(obj, key, list):
    extracted_data = json_extract(obj, key)
    if not extracted_data:
        list.append(None)
    else:
        list.append(extracted_data[0])

In [ ]:
def blocks_data_extract(obj):
    branches = []
    dataviewer_getValue_lists = []
    blocks_names = []
    if 'project' in obj:
        lists = {}
        for stage_actors in obj['project']['targets']:
            if stage_actors['lists']:
                for listID, details in stage_actors['lists'].items():
                    lists[listID] = details[0]

        speed = {4: 'high', 3: 'medium', 2: 'low', 1: 'zero'}
        blocks = []
        for stage_actors in obj['project']['targets']:
            if stage_actors['blocks']:
                for blockID, details in stage_actors['blocks'].items():
                    if isinstance(details, dict) and details['shadow'] == False:
                        blockName = details['opcode']
                        blocks_names.append(blockName)
                        if 'LIST_ID' in details['fields']:
                            listID = details['fields']['LIST_ID'][0]
                            if listID in lists:
                                if blockName == 'dataviewer_getValue' and lists[listID] not in dataviewer_getValue_lists:
                                    dataviewer_getValue_lists.append(lists[listID])
                                blockName = blockName + '(' + lists[listID] + ')'
                            else:
                                blockName = blockName + '(' + listID + ')'
                        if 'WHENTEMPMENU' in details['fields']:
                            blockName = blockName + '(' + details['fields']['WHENTEMPMENU'][0] + ')'
                        if 'PARTICLESPEED' in details['fields']:
                            blockName = blockName + '(' + speed[int(details['fields']['PARTICLESPEED'][0])] + ')'
                        blocks.append((blockID, blockName, details['parent']))
        
        branchesID = []
        while len(blocks) > 0:
            i = 0
            while i < len(blocks):
                if blocks[i][2] == None:
                    branches.append([blocks[i][1]])
                    branchesID.append([blocks[i][0]])
                    del blocks[i]
                    i -= 1
                else:
                    for j in range(0, len(branches)):
                        if blocks[i][2] in branchesID[j]:
                            branches[j].append(blocks[i][1])
                            branchesID[j].append(blocks[i][0])
                            del blocks[i]
                            i -= 1
                            break
                i += 1
    return blocks_names, branches, dataviewer_getValue_lists

In [ ]:
projectID = []
time = []
adjusted_time = []
classname = []
event = []
block_name = []
qty_deleted = []
qty_blocks = []
all_blocks_names = []
all_branches = []
qty_branches = []
all_dataviewer_getValue_lists = []
qty_dataviewer_getValue_lists = []

In [ ]:
path = 'C:\\Users\\Lucas\\Documents\\ProjetoCassia\\Dados'
folders = [name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))]
for folder in folders:
    file_path = path + '\\' + folder + '\\timeline.json'
    if os.path.isfile(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        time_zero = int(next(iter(data))[0:10])
        time_old = time_zero
        time_stopped = 0
        for k in data:
            projectID.append(folder)
            time_new = int(k[0:10]) - time_zero
            time.append(time_new)
            if data[k]['classname'] == 'TIMELINE' and data[k]['event'] == 'projectLoaded*':
                time_stopped += time_new - time_old
            adjusted_time.append(time_new - time_stopped)
            time_old = time_new
            list_append(data[k], 'classname', classname)
            list_append(data[k], 'event', event)
            list_append(data[k], 'block', block_name)
            list_append(data[k], 'deletedBlocks', qty_deleted)
            blocks_data = blocks_data_extract(data[k])
            blocksNames = blocks_data[0]
            if not blocksNames:
                all_blocks_names.append(None)
                qty_blocks.append(None)
            else:
                all_blocks_names.append(blocksNames)
                qty_blocks.append(len(blocksNames))
            branches = blocks_data[1]
            if not branches:
                all_branches.append(None)
                qty_branches.append(None)
            else:
                all_branches.append(branches)
                qty_branches.append(len(branches))
            dataviewer_getValue_lists = blocks_data[2]
            if not dataviewer_getValue_lists:
                all_dataviewer_getValue_lists.append(None)
                qty_dataviewer_getValue_lists.append(None)
            else:
                all_dataviewer_getValue_lists.append(dataviewer_getValue_lists)
                qty_dataviewer_getValue_lists.append(len(dataviewer_getValue_lists))

In [ ]:
df = pd.DataFrame(
    list(zip(projectID, time, adjusted_time, classname, event, block_name, qty_deleted, qty_blocks, all_blocks_names,
             qty_branches, all_branches, qty_dataviewer_getValue_lists, all_dataviewer_getValue_lists)),
    columns = ['projectID', 'time', 'adjusted_time', 'classname', 'event', 'block_name', 'qty_delete', 'qty_blocks',
               'all_blocks_names', 'qty_branches', 'all_branches', 'qty_dataviewer_getValue_lists',
               'all_dataviewer_getValue_lists']
)

df['adjusted_time_min'] = df['adjusted_time'].div(60)

In [ ]:
df = df.sort_values(['projectID', 'time'])

In [ ]:
#Visualização de todos os dados
display_df(df)

In [ ]:
#Visualização de todas as combinações de classname e event
display_df(df[['classname', 'event']].drop_duplicates().sort_values(['classname', 'event']), False)

In [ ]:
#Visualização do estado final de cada projeto
display_df(df.loc[df[(df.event == 'PROJECT_CHANGED')].groupby('projectID').time.idxmax()])

In [ ]:
df_blocks_diff = df[['projectID', 'adjusted_time', 'adjusted_time_min', 'qty_blocks']][(df.event == 'PROJECT_CHANGED')]
df_blocks_diff = df_blocks_diff.reset_index().drop(columns = ['index']).fillna(0)
blocks_diff = []
blocks_diff.append(0)
for i in range(len(df_blocks_diff) - 1):
    if df_blocks_diff.loc[i, 'projectID'] == df_blocks_diff.loc[i + 1, 'projectID']:
        blocks_diff.append(df_blocks_diff.loc[i + 1, 'qty_blocks'] - df_blocks_diff.loc[i, 'qty_blocks'])
    else:
        blocks_diff.append(0)
df_blocks_diff['blocks_diff'] = blocks_diff
df_blocks_deleted = df_blocks_diff[(df_blocks_diff.blocks_diff < 0)]

In [ ]:
#Importante: a ordem da lista (event_list) não é a mesma ordem que os dados são gerados no analysis_count.
event_list = ['PROJECT_RUN_START', 'PROJECT_RUN_STOP', 'PROJECT_START', 'SCRIPT_GLOW_ON', 'SCRIPT_GLOW_OFF', 'greenFlag*',
              'addBackdrop*', 'deleteBackdrop*', 'addCostume*', 'deleteCostume*', 'addSprite*', 'deleteSprite*',
              'targetWasCreated', 'targetWasRemoved']

df_event_count = df[['projectID', 'event']][df.event.isin(event_list)].groupby(['projectID', 'event']).size().unstack(
    fill_value = 0)

classname_list = ['SVGSkin', 'BitmapSkin']

df_classname_count = df[['projectID', 'classname']][df.classname.isin(classname_list)]
df_classname_count = df_classname_count.groupby(['projectID', 'classname']).size().unstack(fill_value = 0)

df_qty_delete = df_blocks_deleted[['projectID', 'blocks_diff']].groupby(['projectID']).agg(['sum'])
df_qty_delete.columns = ['Blocos_Apagados']
df_qty_delete['Blocos_Apagados'] = abs(df_qty_delete['Blocos_Apagados'])

df_qty_blocks_end = df.loc[df[(df.event == 'PROJECT_CHANGED')].groupby('projectID').time.idxmax()][['projectID', 'qty_blocks']]
df_qty_blocks_end.set_index('projectID', inplace = True)
df_qty_blocks_end.columns = ['Qtde_Blocos_Fim']

df_qty_branches_end = df.loc[df[(df.event == 'PROJECT_CHANGED')].groupby('projectID').time.idxmax()]
df_qty_branches_end = df_qty_branches_end[['projectID', 'qty_branches']]
df_qty_branches_end.set_index('projectID', inplace = True)
df_qty_branches_end.columns = ['Qtde_Ramos_Fim']

df_total_time = df[['projectID', 'time']].groupby(['projectID']).agg(['max'])
df_total_time.columns = ['Tempo_Total']

df_adjusted_time = df[['projectID', 'adjusted_time']].groupby(['projectID']).agg(['max'])
df_adjusted_time.columns = ['Tempo_Ajustado']

df_adjusted_time_min = df[['projectID', 'adjusted_time_min']].groupby(['projectID']).agg(['max'])
df_adjusted_time_min.columns = ['Tempo_Ajustado (min)']

analysis_dfs = [df_event_count, df_classname_count, df_qty_delete, df_qty_blocks_end, df_qty_branches_end, df_total_time,
                df_adjusted_time, df_adjusted_time_min]

analysis = reduce(lambda left, right: pd.merge(left, right, left_index = True, right_index = True, how = 'outer'), analysis_dfs)
analysis = analysis.fillna(0)

#Ordenando todas as colunas possíveis
ordered_cols = ['PROJECT_RUN_START', 'PROJECT_RUN_STOP', 'SCRIPT_GLOW_ON', 'SCRIPT_GLOW_OFF', 'PROJECT_START', 'greenFlag*',
                'targetWasCreated', 'targetWasRemoved', 'Blocos_Apagados', 'Qtde_Blocos_Fim', 'Qtde_Ramos_Fim', 'Tempo_Total',
                'Tempo_Ajustado', 'Tempo_Ajustado (min)', 'SVGSkin', 'BitmapSkin', 'addBackdrop*', 'deleteBackdrop*',
                'addCostume*', 'deleteCostume*', 'addSprite*', 'deleteSprite*']

#Selecionando as colunas existentes no DataFrame analysis
analysis_ordered_cols = []
for col in ordered_cols:
    if col in analysis:
        analysis_ordered_cols.append(col)

analysis = analysis[analysis_ordered_cols]

In [ ]:
display_df(analysis)

In [ ]:
# if os.path.isfile(path + '\\analysis.csv'):
#     msgBox = ctypes.windll.user32.MessageBoxW(0, "Deseja sobrescrever o arquivo existente?", "Alerta", 4)
# if msgBox == 6:
#     analysis.to_csv(path + '\\analysis.csv')

analysis.to_csv(path + '\\analysis.csv')

# analysis.to_excel(path + '\\analysis.xlsx')

In [ ]:
df_time_diff = df[['projectID', 'adjusted_time', 'adjusted_time_min', 'event']][(df.event == 'PROJECT_RUN_START') | (
    df.event == 'PROJECT_RUN_STOP')].reset_index().drop(columns = ['index'])
time_diff = []
for i in range(len(df_time_diff) - 1):
    if df_time_diff.loc[i, 'projectID'] == df_time_diff.loc[i + 1, 'projectID']:
        time_diff.append(int(df_time_diff.loc[i + 1, 'adjusted_time']) - int(df_time_diff.loc[i, 'adjusted_time']))
    else:
        time_diff.append(0)
time_diff.append(0)
df_time_diff['time_diff'] = time_diff

In [ ]:
df_events_table = pd.DataFrame()
df_events_backdrop = pd.DataFrame()
df_events_addbackdrop = pd.DataFrame()
df_events_deletebackdrop = pd.DataFrame()
df_events_addcostume = pd.DataFrame()
df_events_deletecostume = pd.DataFrame()
df_events_addsprite = pd.DataFrame()
df_events_deletesprite = pd.DataFrame()
df_events_run_zero = pd.DataFrame()
df_events_run_not_zero = pd.DataFrame()
df_events_addblock = pd.DataFrame()
df_events_deleteblock = pd.DataFrame()
df_blocks = pd.DataFrame()
df_events_blocks = pd.DataFrame()
df_events_blockdrag = pd.DataFrame()

def fill_df_events():
    global df_events_table
    global df_events_backdrop
    global df_events_addbackdrop
    global df_events_deletebackdrop
    global df_events_addcostume
    global df_events_deletecostume
    global df_events_addsprite
    global df_events_deletesprite
    global df_events_run_zero
    global df_events_run_not_zero
    global df_events_addblock
    global df_events_deleteblock
    global df_blocks
    global df_events_blocks
    global df_events_blockdrag

    time = 'adjusted_time'
    if minutes:
        time += '_min'
    
    df_ID = df[df.projectID == selected_projectID]

    df_time_event = df_ID[[time, 'event']]
    df_time_event.columns = ['time', 'data']
    
    df_time_classname = df_ID[[time, 'classname']]
    df_time_classname.columns = ['time', 'data']
    
    df_events_table = df_time_event[(df_time_event.data == 'openDataviewerTable*') |
                                    (df_time_event.data == 'closeDataviewerTable*')]
    df_events_table = df_events_table.replace(['openDataviewerTable*', 'closeDataviewerTable*'], [bar_height, 0])

    df_events_backdrop = df_time_classname.replace(['SVGSkin', 'BitmapSkin'], bar_height)
    df_events_backdrop.loc[~df_events_backdrop['data'].isin([bar_height]), 'data'] = 0

    df_events_addbackdrop = df_time_event[(df_time_event.data == 'addBackdrop*')]
    df_events_addbackdrop = df_events_addbackdrop.replace('addBackdrop*', bar_height).drop_duplicates()

    df_events_deletebackdrop = df_time_event[(df_time_event.data == 'deleteBackdrop*')]
    df_events_deletebackdrop = df_events_deletebackdrop.replace('deleteBackdrop*', bar_height).drop_duplicates()

    df_events_addcostume = df_time_event[(df_time_event.data == 'addCostume*')]
    df_events_addcostume = df_events_addcostume.replace('addCostume*', bar_height).drop_duplicates()

    df_events_deletecostume = df_time_event[(df_time_event.data == 'deleteCostume*')]
    df_events_deletecostume = df_events_deletecostume.replace('deleteCostume*', bar_height).drop_duplicates()

    df_events_addsprite = df_time_event[(df_time_event.data == 'addSprite*')]
    df_events_addsprite = df_events_addsprite.replace('addSprite*', bar_height).drop_duplicates()

    df_events_deletesprite = df_time_event[(df_time_event.data == 'deleteSprite*') |
                                           (df_time_event.data == 'targetWasRemoved*')]
    df_events_deletesprite = df_events_deletesprite.replace(['deleteSprite*', 'targetWasRemoved*'], bar_height)
    df_events_deletesprite = df_events_deletesprite.drop_duplicates()

    df_ID_td = df_time_diff[df_time_diff.projectID == selected_projectID]
    
    df_events_run_zero = df_ID_td[(df_ID_td.event == 'PROJECT_RUN_START') & (df_ID_td.time_diff == 0)][[time, 'time_diff']]
    df_events_run_zero = df_events_run_zero.drop_duplicates()
    df_events_run_zero['time_diff'] = bar_height
    df_events_run_zero.columns = ['time', 'data']
    
    df_events_run_not_zero = df_ID_td[((df_ID_td.event == 'PROJECT_RUN_START') & (df_ID_td.time_diff != 0)) |
                                      (df_ID_td.event == 'PROJECT_RUN_STOP')][[time, 'event']]
    df_events_run_not_zero = df_events_run_not_zero[(df_events_run_not_zero.event == 'PROJECT_RUN_START') | 
                                                    (df_events_run_not_zero.event == 'PROJECT_RUN_STOP')]
    df_events_run_not_zero = df_events_run_not_zero.replace(['PROJECT_RUN_START', 'PROJECT_RUN_STOP'], [bar_height, 0])
    df_events_run_not_zero.columns = ['time', 'data']
    
    df_events_addblock = df_time_event[(df_time_event.data == 'createBlock*')]
    df_events_addblock = df_events_addblock.replace('createBlock*', bar_height).drop_duplicates()

    df_events_deleteblock = df_blocks_deleted[df_blocks_deleted.projectID == selected_projectID][[time, 'blocks_diff']]
    df_events_deleteblock['blocks_diff'] = bar_height
    df_events_deleteblock.columns = ['time', 'data']

    df_blocks = df_ID[[time, 'qty_blocks']].dropna()
    df_blocks_zero = pd.DataFrame({time: 0, 'qty_blocks': 0}, index = [0])
    df_events_blocks = pd.concat([df_blocks_zero, df_blocks])
    df_events_blocks.columns = ['time', 'data']

    df_events_blockdrag = df_time_event[(df_time_event.data == 'BLOCK_DRAG_UPDATE')]
    df_events_blockdrag = df_events_blockdrag.replace('BLOCK_DRAG_UPDATE', bar_height).drop_duplicates()

In [ ]:
selected_projectID = '1'
bar_height = 5
minutes = False

bar_width = 1 #largura da barra - vai variar dependendo do tempo total do arquivo. Sugestão: 0.01 para min e 1 para sec
if minutes:
    bar_width = 0.01

time = 'adjusted_time'
if minutes:
    time += '_min'
end_time = df[df.projectID == selected_projectID][['projectID', time]].groupby(['projectID']).agg(['max']).iloc[0]

fill_df_events()

subplot_titles = ['Editing Backdrop/Costume', 'Project Running', 'Blocks Added/Deleted', 'Blocks Used', 'Block Drag']

fig = go.Figure()
fig = make_subplots(
    rows = 5,
    cols = 1,
    shared_xaxes = True,
    vertical_spacing = 0.1,
    subplot_titles = (subplot_titles))

fig.add_trace(
    go.Scatter(
        x = df_events_backdrop.time,
        y = df_events_backdrop.data,
        name = 'Editing Backdrop/Costume',
        mode = 'lines',
        fill = "tozeroy",
        line = {'color': 'midnightblue', 'width': 0, 'shape': 'hv'}), #shape hv define que é uma linha do tipo step
    row = 1, col = 1)

fig.add_trace(
    go.Scatter(
        x = df_events_table.time,
        y = df_events_table.data,
        mode = 'lines',
        name = 'Table open',
        fill="tozeroy",
        line={'color': 'seagreen', 'width': 0, 'shape': 'hv'}), 
    row = 1, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_addbackdrop.time,
        y = df_events_addbackdrop.data, 
        width = bar_width,
        marker = {'line': {'width': 0}}, #removendo a borda dos marcadores
        name = 'Backdrop added'), 
    row = 1, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_deletebackdrop.time,
        y = df_events_deletebackdrop.data,
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Backdrop deleted'), 
    row = 1, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_addcostume.time,
        y = df_events_addcostume.data, 
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Costume added'), 
    row = 1, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_deletecostume.time,
        y = df_events_deletecostume.data,
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Costume deleted'), 
    row = 1, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_addsprite.time,
        y = df_events_addsprite.data, 
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Sprite added'), 
    row = 1, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_deletesprite.time,
        y = df_events_deletesprite.data,
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Sprite deleted'), 
    row = 1, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_run_zero.time,
        y = df_events_run_zero.data,
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Running (0s)'), 
    row = 2, col = 1)

fig.add_trace(
    go.Scatter(
        x = df_events_run_not_zero.time,
        y = df_events_run_not_zero.data,
        mode = 'lines',
        name = 'Running (>0s)',
        fill = "tozeroy",
        line = {'width': 0, 'shape': 'hv'}), 
    row = 2, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_addblock.time,
        y = df_events_addblock.data,
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Block added'), 
    row = 3, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_deleteblock.time,
        y = df_events_deleteblock.data,
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Block deleted'), 
    row = 3, col = 1)

fig.add_trace(
    go.Scatter(
        x = df_events_blocks.time,
        y = df_events_blocks.data,
        #mode = 'lines+markers',
        name = 'Blocks used',
        fill = "tozeroy",
        line = {'color': 'lightsteelblue', 'width': 0, 'shape': 'hv'}),
    row = 4, col = 1)

fig.add_trace(
    go.Bar(
        x = df_events_blockdrag.time,
        y = df_events_blockdrag.data,
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Block drag'),
    row = 5, col = 1)

fig.update_layout(
    showlegend = True,
    plot_bgcolor = 'rgba(0,0,0,0)',
    #legend={
    #    'orientation': 'h',
    #    'yanchor': 'bottom',
    #    'y': -0.4,
    #    'xanchor': 'center',
    #    'x': 0.5},
    font = {'size': 8},
    title = {
        'text': 'ProjectID ' + selected_projectID,
        'font_size': 20,
        'xanchor': 'center',
        'x': 0.5})

fig.update_xaxes(range=[0, int(end_time)])

fig.update_yaxes(showticklabels = False, row = 1, col = 1)
fig.update_yaxes(showticklabels = False, row = 2, col = 1)
fig.update_yaxes(showticklabels = False, row = 3, col = 1)
fig.update_yaxes(showticklabels = False, row = 5, col = 1)

#Arrumando a posição dos títulos dos subplots
for i in range(5):
    fig.layout.annotations[i].update(xanchor='left')
    fig.layout.annotations[i].update(x=-0.02)

fig.show()

In [ ]:
#ajustando para as barras ficarem na mesma altura que o maior número de blocos usados
max_blocks = df_blocks.qty_blocks.max()
size_adjust = max_blocks / bar_height
df_events_backdrop['data'] = df_events_backdrop['data'] * size_adjust
df_events_run_zero['data'] = df_events_run_zero['data'] * size_adjust
df_events_run_not_zero['data'] = df_events_run_not_zero['data'] * size_adjust

In [ ]:
fig2 = go.Figure()

fig2.add_trace(
    go.Scatter(
        x = df_events_blocks.time,
        y = df_events_blocks.data,
        #mode = 'lines+markers',
        name = 'Blocks used',
        line = {'color': 'black', 'width': 2, 'shape': 'hv'}))

fig2.add_trace(
    go.Scatter(
        x = df_events_backdrop.time,
        y = df_events_backdrop.data,
        name = 'Editing backdrop or costume',
        mode = 'lines',
        fill = "tozeroy",
        line = {'width': 0, 'shape': 'hv'}))

fig2.add_trace(
    go.Bar(
        x = df_events_run_zero.time,
        y = df_events_run_zero.data,
        width = bar_width,
        marker = {'line': {'width': 0}},
        name = 'Running (0s)'))

fig2.add_trace(
    go.Scatter(
        x = df_events_run_not_zero.time,
        y = df_events_run_not_zero.data,
        mode = 'lines',
        name = 'Running (>0s)',
        fill = "tozeroy",
        line = {'width': 0, 'shape': 'hv'}))

fig2.update_layout(
    showlegend = True, 
    plot_bgcolor = 'rgba(0,0,0,0)',
    title = {
        'text': 'ProjectID ' + selected_projectID,
        'font_size': 20,
        'xanchor': 'center',
        'x': 0.5})

In [ ]:
#precisa instalar o kaleido, pode ser via pip ou via conda:
#via pip
#pip install -U kaleido
#via conda
#conda install -c conda-forge python-kaleido
fig.write_image(path + '\\graficos\\grafico_' + selected_projectID + '.svg')
fig2.write_image(path + '\\graficos\\grafico2_' + selected_projectID + '.svg')

In [ ]:
projects_list = ['1', '5', '9']

subplot_titles = []
for project in projects_list:
    subplot_titles.append('ProjectID ' + project)

fig3 = go.Figure()
fig3 = make_subplots(
    rows = len(projects_list),
    cols = 1,
    shared_xaxes = True,
    vertical_spacing = 0.1,
    subplot_titles = (subplot_titles))

i = 1
for project in projects_list:
    
    selected_projectID = project
    fill_df_events()
    add_legends = False
    if i == 1:
        add_legends = True
    
    
    max_blocks = df_blocks.qty_blocks.max()
    size_adjust = max_blocks / bar_height
    df_events_backdrop['data'] = df_events_backdrop['data'] * size_adjust
    df_events_run_zero['data'] = df_events_run_zero['data'] * size_adjust
    df_events_run_not_zero['data'] = df_events_run_not_zero['data'] * size_adjust
    
    fig3.add_trace(
        go.Scatter(
            x = df_events_blocks.time,
            y = df_events_blocks.data,
            #mode = 'lines+markers',
            name = 'Blocks used',
            line = {'color': 'black', 'width': 2, 'shape': 'hv'},
            showlegend = add_legends),
    row = i, col = 1)

    fig3.add_trace(
        go.Scatter(
            x = df_events_backdrop.time,
            y = df_events_backdrop.data,
            name = 'Editing backdrop or costume',
            mode = 'lines',
            fill = "tozeroy",
            line = {'color': 'red', 'width': 0, 'shape': 'hv'},
            showlegend = add_legends),
    row = i, col = 1)

    fig3.add_trace(
        go.Bar(
            x = df_events_run_zero.time,
            y = df_events_run_zero.data,
            width = bar_width,
            marker = {'line': {'width': 0}, 'color': 'green'},
            name = 'Running (0s)',
            showlegend = add_legends),
    row = i, col = 1)

    fig3.add_trace(
        go.Scatter(
            x = df_events_run_not_zero.time,
            y = df_events_run_not_zero.data,
            mode = 'lines',
            name = 'Running (>0s)',
            fill = "tozeroy",
            line = {'color': 'blue', 'width': 0, 'shape': 'hv'},
            showlegend = add_legends),
    row = i, col = 1)
    
    i += 1

fig3.update_layout(
    showlegend=True, 
    plot_bgcolor='rgba(0,0,0,0)',
    font={'size': 8},
    title={
        'text': 'Comparing Projects',
        'font_size': 20,
        'xanchor': 'center',
        'x': 0.5})

for i in range(len(projects_list)):
    fig3.layout.annotations[i].update(xanchor='left')
    fig3.layout.annotations[i].update(x=-0.02)

fig3.show()

In [ ]:
import glob
import re
import base64

In [ ]:
#Adicionando as imagens (screenshots) ao gráfico
data_folder = Path('C:\\Users\\Lucas\\Documents\\ProjetoCassia\\Dados\\' + selected_projectID)

list_images =  []
list_new_images = []

file_images = glob.glob(os.path.join(data_folder, '*.jpg')) #adiciona todas as imagens da pasta a uma lista

#print (file_images)
for fname in file_images:
    res = re.findall("timeline_(\d+).jpg", fname)
    if not res: continue
    file_new_images = res[0] #res[0] é o número da cada arquivo de imagem, correspondente ao tempo em que foi salvo
    list_images.append(res[0]) 
    new_image_time = int(file_new_images[0:10]) - time_zero
    list_new_images.append(new_image_time) 
#print (list_images)

#adding local images to the plot
#adicionando múltiplas imagens - uma para cada run
i=0
length = len(list_images)
for i in range(length):
    image_filename = file_images[i]  #nome da imagem
    screenshot = base64.b64encode(open(data_folder / image_filename, 'rb').read())
    fig.add_layout_image(dict(
        source='data:image/jpg;base64,{}'.format(screenshot.decode()),
        x=list_new_images[i], 
        y=1,
        xref="x",
        yref="paper",
        )
    )

fig.update_layout_images(dict(
        sizex= 10,#(time_events[-1])/8, #tamanho da imagem - tempo total/10
        sizey= 4, # time_events[-1])/10, 
        xanchor="right",
        yanchor="bottom"
))